In [ ]:
import random
import json
import time
from web3 import Web3
from solcx import compile_standard, install_solc
from web3.exceptions import ContractLogicError, BadFunctionCallOutput
########################################################################
# 0) Setup & Reproducible Random
########################################################################
# Set a fixed seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

########################################################################
# Connect to Ganache & Basic Setup
########################################################################
GANACHE_URL = "http://127.0.0.1:8545"
w3 = Web3(Web3.HTTPProvider(GANACHE_URL))
assert w3.is_connected(), "Could not connect to Ganache"

accounts = w3.eth.accounts
publisher = accounts[0]

# Define deployment parameters
deadline_seconds = 60  # 10 minutes from now

U = 99  # attempt U "user"
# We'll attempt 15 "users" (some fail). At least 11 accounts are needed for 15 attempts.
# If you launched Ganache with `ganache -a 20` or similar, you have enough. For example:
possible_users = accounts[1:U+1]  # U+1 attempts
print(f"Publisher: {publisher}")
print(f"Attempting participants: {possible_users}")


Publisher: 0xAcA7d2813dA5FF0146429803c4FcB8F5c4086DE3
Attempting participants: ['0x85bD17E7a0B9C96dc14470c69d9CeE6617F83aaB', '0x8E6670a1E36Ba00Bc2b21B6687C54447616A48Ef', '0x557598fd2fC58542f34C88746b1A479Ba3d1d753', '0x6653DBeDd9546a8B6c510778caC65b4474A21949', '0xB3c82BFAb7b450f1B13752071e6AaaB18b84556D', '0xC37D7889ad62de4Fc5798EC7A6fc8dfE6764BBDB', '0x92942835078BbFa9D68A7106aaE6d61a7EB557a2', '0x1cF8d0dA1062975273bBF853b8Aa3221CeF61608', '0xCA1368eFE9b8aE916368F1cA229283a712C0Cd26', '0x658b64D0B8faA91c81A353a11bdbB258cF3E4cAb', '0x7F728dFEA7FE0db60C7f664F5D96c077535e4dBf', '0x3bC80Dd91EF7dD35f8F6CDa6Ea5dEcd2eAF29679', '0xaE45f58c7c27020c341153A5caCFd6dA295d510D', '0x1af38d7bB2de936107ca004c24f5D62E3ac194FF', '0xd7C918590253605939367FC34c499932fDEEEF10', '0x213AAB4B4752921E933E1B118B27d4167b903292', '0x89022b2e9CbA63b7c6F1AeF5E8b83d1F92896f9F', '0xC4a76b530Ff27A469AA2fEd90f43057e8AE065B1', '0xD8EeD91e800c88D017722Fd575fED348E611C844', '0xBaeF5dB9044638E3A689dd6a03C0B2F3e0666cEa', 

In [9]:
########################################################################
# 1) Compile the FLIncentiveMultiCat contract
########################################################################
with open("../contracts/FLIncentiveMultiCat.sol", "r") as f:
    contract_source_code = f.read()
# 2) Install the required version of solc
install_solc('0.8.17')

# 3) Compile using solcx
#    The compile_standard function expects a specific JSON format.
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "FLIncentiveMultiCat.sol": {"content": contract_source_code}
        },
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["metadata", "evm.bytecode", "evm.deployedBytecode", "abi"]
                }
            }
        }
    },
    solc_version="0.8.17",
)

contract_info = compiled_sol["contracts"]["FLIncentiveMultiCat.sol"]["FLIncentiveMultiCat"]
abi = contract_info["abi"]
bytecode = contract_info["evm"]["bytecode"]["object"]

print("Contract compiled successfully.\n")


Contract compiled successfully.



In [3]:
# ---------------------------------------------------------------------
# 3) Deploy the Contract
# ---------------------------------------------------------------------

# Create contract object
contract = w3.eth.contract(abi=abi, bytecode=bytecode)

# Define deployment parameters
deadline_seconds = 100  # 10 minutes from now
deadline_timestamp = int(time.time()) + deadline_seconds
reward_pool_eth = 10  # 10 ETH as the reward pool

# Build deployment transaction
deploy_txn = contract.constructor(deadline_timestamp).build_transaction({
    "from": publisher,
    "value": w3.to_wei(reward_pool_eth, "ether"),  # 10 ETH in Wei
    "nonce": w3.eth.get_transaction_count(publisher),
    # "gas": 5000000,  # Adjusted Gas Limit
    "gasPrice": w3.to_wei("1", "gwei"),
})

# Sign and send the deployment transaction
try:
    # Since Ganache automatically unlocks accounts, no need to sign manually
    deploy_hash = w3.eth.send_transaction(deploy_txn)
    print("Deploying contract...")
    deploy_receipt = w3.eth.wait_for_transaction_receipt(deploy_hash)
    
    # Check deployment status
    if deploy_receipt.status != 1:
        print("Contract deployment failed.")
        # Optionally, attempt to extract revert reason
        # revert_reason = get_revert_reason(deploy_hash)
        # print(f"Revert Reason: {revert_reason}")
    else:
        print("Contract deployed successfully.")
    
    # Retrieve contract address
    contract_address = deploy_receipt.contractAddress
    if not contract_address:
        print("Failed to retrieve contract address.")
        exit(1)
    else:
        print(f"Contract deployed at address: {contract_address}\n")
    
    # Initialize contract instance
    contract_instance = w3.eth.contract(address=contract_address, abi=abi)
    
    # Fetch and display deadline information
    on_chain_deadline = contract_instance.functions.deadline().call()
    current_time = int(time.time())
    time_until_deadline = on_chain_deadline - current_time
    print(f"On-chain deadline: {on_chain_deadline} (current time: {current_time})")
    print(f"Time until deadline: {time_until_deadline} seconds\n")

except ContractLogicError as e:
    print(f"ContractLogicError during deployment: {e}")
except BadFunctionCallOutput as e:
    print(f"BadFunctionCallOutput during deployment: {e}")
except Exception as e:
    print(f"An unexpected error occurred during deployment: {e}")
    exit(1)


Deploying contract...
Contract deployed successfully.
Contract deployed at address: 0xf5E9131bb41A4c85fd498cBC081d4DabEd2ef532

On-chain deadline: 1737569144 (current time: 1737569044)
Time until deadline: 100 seconds



In [4]:
########################################################################
# 3) Assign Scenarios to All Users
########################################################################
# Define probability thresholds as constants
# INVALID_SYNERGY_PROB = 0.1       # 10% chance
# INCORRECT_STAKE_PROB = 0.2       # 20% chance (0.2 <= r < 0.4)
# FORCED_LATE_JOIN_PROB = 0.2      # 20% chance (0.4 <= r < 0.5)
# NORMAL_SUCCESS_PROB = 0.5         # 50% chance (r >= 0.5)

INVALID_SYNERGY_PROB = 0.       # 10% chance
INCORRECT_STAKE_PROB = 0.       # 20% chance (0.2 <= r < 0.4)
FORCED_LATE_JOIN_PROB = 0.      # 10% chance (0.4 <= r < 0.5)
NORMAL_SUCCESS_PROB = 1.         # 60% chance (r >= 0.6)

def random_cat_in_range() -> int:
    """
    Returns a valid data/resource category [0, 1, 2].
    0 = Low, 1 = Medium, 2 = High
    """
    return random.randint(0, 2)

def random_cat_out_of_range() -> int:
    """
    Returns an invalid category (3), expected to cause the transaction to revert.
    """
    return 3

def determine_scenario(r: float) -> str:
    """
    Determines the join attempt scenario based on the random value r.
    
    Parameters:
        r (float): A random float between 0 and 1.
    
    Returns:
        str: The scenario name.
    """
    if r < INVALID_SYNERGY_PROB:
        return "Invalid Synergy"
    elif r < INVALID_SYNERGY_PROB + INCORRECT_STAKE_PROB:
        return "Incorrect Stake"
    elif r < INVALID_SYNERGY_PROB + INCORRECT_STAKE_PROB + FORCED_LATE_JOIN_PROB:
        return "Forced Late Join"
    else:
        return "Normal Success"

def get_combo_params(dcat: int, rcat: int) -> tuple:
    """
    Fetches the stake required and reward multiplier for the given data and resource categories.
    
    Parameters:
        dcat (int): Data category (0=Low, 1=Medium, 2=High)
        rcat (int): Resource category (0=Low, 1=Medium, 2=High)
    
    Returns:
        tuple: (stakeRequired, rewardMultiplier)
    """
    return contract_instance.functions.getComboParams(dcat, rcat).call()

# Assign scenarios to all users
user_scenarios = []
for acct in possible_users:
    r = random.random()
    scenario = determine_scenario(r)
    
    if scenario == "Invalid Synergy":
        dcat = random_cat_in_range()
        rcat = random_cat_out_of_range()
    elif scenario == "Incorrect Stake":
        dcat = random_cat_in_range()
        rcat = random_cat_in_range()
    elif scenario == "Forced Late Join":
        dcat = random_cat_in_range()
        rcat = random_cat_in_range()
    else:  # "Normal Success"
        dcat = random_cat_in_range()
        rcat = random_cat_in_range()
    
    synergy_info = (dcat, rcat)
    user_scenarios.append({
        "address": acct,
        "scenario": scenario,
        "synergy": synergy_info
    })

########################################################################
# 4) Separate Users into Groups Based on Scenario
########################################################################
# Group users by scenario
normal_success_users = []
invalid_synergy_users = []
incorrect_stake_users = []
forced_late_join_users = []

for user in user_scenarios:
    if user["scenario"] == "Normal Success":
        normal_success_users.append(user)
    elif user["scenario"] == "Invalid Synergy":
        invalid_synergy_users.append(user)
    elif user["scenario"] == "Incorrect Stake":
        incorrect_stake_users.append(user)
    elif user["scenario"] == "Forced Late Join":
        forced_late_join_users.append(user)

# Initialize lists to track outcomes
successful_users = []
fail_reasons = []


In [5]:

########################################################################
# 5) Process Normal Success, Invalid Synergy, and Incorrect Stake Attempts
########################################################################
print("Processing Normal Success, Invalid Synergy, and Incorrect Stake join attempts...\n")

for i, user in enumerate(normal_success_users + invalid_synergy_users + incorrect_stake_users, start=1):
    acct = user["address"]
    scenario = user["scenario"]
    synergy_info = user["synergy"]
    dcat, rcat = synergy_info
    
    print(f"User Attempt {i}: Scenario={scenario}, Synergy={synergy_info}")
    
    try:
        # Fetch required stake from the contract; may revert if synergy is invalid
        stake_req, _ = get_combo_params(dcat, rcat)
        
        # Modify stake if the scenario is "Incorrect Stake"
        if scenario == "Incorrect Stake":
            stake_req = stake_req // 2 if stake_req > 0 else 1  # Halve the stake or set to 1 Wei
        
        # For "Normal Success" and "Invalid Synergy", no time manipulation is needed
        # Build the join transaction
        join_txn = contract_instance.functions.join(dcat, rcat).build_transaction({
            "from": acct,
            "value": stake_req,
            "nonce": w3.eth.get_transaction_count(acct),
            "gas": 3_000_000,
            "gasPrice": w3.to_wei("1", "gwei"),
        })
        
        # Send the transaction
        j_hash = w3.eth.send_transaction(join_txn)
        j_receipt = w3.eth.wait_for_transaction_receipt(j_hash)
        
        # After transaction is mined, verify if the participant joined successfully
        part_data = contract_instance.functions.participants(acct).call()
        joined_status = part_data[0]
        
        if joined_status:
            # Append to successful_users with relevant details
            successful_users.append({
                "address": acct,
                "synergy": synergy_info,
                "scenario": scenario,
                "stake": stake_req
            })
            print(f"  SUCCESS: Joined successfully with synergy {synergy_info}, Stake={stake_req} Wei.\n")
        else:
            # If `joined` is False, log as a failure
            fail_reasons.append({
                "address": acct,
                "synergy": synergy_info,
                "scenario": scenario,
                "reason": "Transaction did not set joined=True"
            })
            print(f"  FAIL: Transaction did not set joined=True.\n")
    
    except Exception as e:
        # Log any exceptions as failures with error messages
        fail_reasons.append({
            "address": acct,
            "synergy": synergy_info,
            "scenario": scenario,
            "reason": str(e)
        })
        print(f"  FAIL: Error occurred - {str(e)[:100]}...\n")


Processing Normal Success, Invalid Synergy, and Incorrect Stake join attempts...

User Attempt 1: Scenario=Normal Success, Synergy=(0, 2)
  SUCCESS: Joined successfully with synergy (0, 2), Stake=3000000000000000000 Wei.

User Attempt 2: Scenario=Normal Success, Synergy=(0, 0)
  SUCCESS: Joined successfully with synergy (0, 0), Stake=1000000000000000000 Wei.

User Attempt 3: Scenario=Normal Success, Synergy=(2, 2)
  SUCCESS: Joined successfully with synergy (2, 2), Stake=4000000000000000000 Wei.

User Attempt 4: Scenario=Normal Success, Synergy=(0, 2)
  SUCCESS: Joined successfully with synergy (0, 2), Stake=3000000000000000000 Wei.

User Attempt 5: Scenario=Normal Success, Synergy=(0, 0)
  SUCCESS: Joined successfully with synergy (0, 0), Stake=1000000000000000000 Wei.

User Attempt 6: Scenario=Normal Success, Synergy=(2, 2)
  SUCCESS: Joined successfully with synergy (2, 2), Stake=4000000000000000000 Wei.

User Attempt 7: Scenario=Normal Success, Synergy=(0, 2)
  SUCCESS: Joined succ

In [6]:

########################################################################
# 6) Process Forced Late Join Attempts
########################################################################
if forced_late_join_users:
    print("Processing Forced Late Join attempts by advancing blockchain time...\n")
    
    # Calculate total time to advance to pass the deadline
    on_chain_deadline = contract_instance.functions.deadline().call()
    current_ts = w3.eth.get_block("latest")["timestamp"]
    leftover = on_chain_deadline - current_ts
    
    if leftover > 0:
        # Advance time by the leftover plus an extra second to ensure deadline is passed
        time_to_add = leftover + 1
        print(f"Advancing blockchain time by {time_to_add} seconds to pass deadline.")
        
        # Increase blockchain time
        w3.provider.make_request('evm_increaseTime', [time_to_add])
        
        # Mine a new block to apply the time change
        w3.provider.make_request('evm_mine', [])
        print(f"Blockchain time advanced by {time_to_add} seconds.\n")
    else:
        print("Deadline has already passed.\n")
    
    for i, user in enumerate(forced_late_join_users, start=1):
        acct = user["address"]
        scenario = user["scenario"]
        synergy_info = user["synergy"]
        dcat, rcat = synergy_info
        
        print(f"Forced Late Join Attempt {i}: Scenario={scenario}, Synergy={synergy_info}")
        
        try:
            # Fetch required stake from the contract
            stake_req, _ = get_combo_params(dcat, rcat)
            
            # No stake modification needed for Forced Late Join
            # Build the join transaction
            join_txn = contract_instance.functions.join(dcat, rcat).build_transaction({
                "from": acct,
                "value": stake_req,
                "nonce": w3.eth.get_transaction_count(acct),
                "gas": 3_000_000,
                "gasPrice": w3.to_wei("1", "gwei"),
            })
            
            # Send the transaction
            j_hash = w3.eth.send_transaction(join_txn)
            j_receipt = w3.eth.wait_for_transaction_receipt(j_hash)
            
            # After transaction is mined, verify if the participant joined successfully
            part_data = contract_instance.functions.participants(acct).call()
            joined_status = part_data[0]
            
            if joined_status:
                # Append to successful_users with relevant details
                successful_users.append({
                    "address": acct,
                    "synergy": synergy_info,
                    "scenario": scenario,
                    "stake": stake_req
                })
                print(f"  SUCCESS: Joined successfully with synergy {synergy_info}, Stake={stake_req} Wei.\n")
            else:
                # If `joined` is False, log as a failure
                fail_reasons.append({
                    "address": acct,
                    "synergy": synergy_info,
                    "scenario": scenario,
                    "reason": "Transaction did not set joined=True"
                })
                print(f"  FAIL: Transaction did not set joined=True.\n")
        
        except Exception as e:
            # Log any exceptions as failures with error messages
            fail_reasons.append({
                "address": acct,
                "synergy": synergy_info,
                "scenario": scenario,
                "reason": str(e)
            })
            print(f"  FAIL: Error occurred - {str(e)[:100]}...\n")



In [7]:
########################################################################
# 7) Finalize the Contract to Distribute Rewards
########################################################################
print("Finalizing the contract to distribute rewards...\n")

try:
    # Build the finalize transaction
    finalize_txn = contract_instance.functions.finalize().build_transaction({
        "from": publisher,
        "nonce": w3.eth.get_transaction_count(publisher),
        "gas": 3_000_000,
        "gasPrice": w3.to_wei("1", "gwei"),
    })
    
    # Send the finalize transaction
    finalize_hash = w3.eth.send_transaction(finalize_txn)
    finalize_receipt = w3.eth.wait_for_transaction_receipt(finalize_hash)
    
    print("Contract has been finalized and rewards distributed.\n")
    
except Exception as e:
    print(f"FAIL: Finalize failed - {str(e)[:100]}...\n")

########################################################################
# 8) Summary of Join Attempts
########################################################################
print("Summary of Join Attempts:")
print("===================================")

print("\nSuccessful Participants:")
for idx, user in enumerate(successful_users, start=1):
    print(f"{idx}. Address: {user['address']}")
    print(f"   Scenario: {user['scenario']}")
    print(f"   Synergy: DataCat={user['synergy'][0]}, ResourceCat={user['synergy'][1]}")
    print(f"   Stake Paid: {user['stake']} Wei\n")

print("\nFailed Attempts:")
for idx, failure in enumerate(fail_reasons, start=1):
    print(f"{idx}. Address: {failure['address']}")
    print(f"   Scenario: {failure['scenario']}")
    print(f"   Synergy Attempted: DataCat={failure['synergy'][0]}, ResourceCat={failure['synergy'][1]}")
    print(f"   Reason: {failure['reason']}\n")



Finalizing the contract to distribute rewards...

Contract has been finalized and rewards distributed.

Summary of Join Attempts:

Successful Participants:
1. Address: 0x85bD17E7a0B9C96dc14470c69d9CeE6617F83aaB
   Scenario: Normal Success
   Synergy: DataCat=0, ResourceCat=2
   Stake Paid: 3000000000000000000 Wei

2. Address: 0x8E6670a1E36Ba00Bc2b21B6687C54447616A48Ef
   Scenario: Normal Success
   Synergy: DataCat=0, ResourceCat=0
   Stake Paid: 1000000000000000000 Wei

3. Address: 0x557598fd2fC58542f34C88746b1A479Ba3d1d753
   Scenario: Normal Success
   Synergy: DataCat=2, ResourceCat=2
   Stake Paid: 4000000000000000000 Wei

4. Address: 0x6653DBeDd9546a8B6c510778caC65b4474A21949
   Scenario: Normal Success
   Synergy: DataCat=0, ResourceCat=2
   Stake Paid: 3000000000000000000 Wei

5. Address: 0xB3c82BFAb7b450f1B13752071e6AaaB18b84556D
   Scenario: Normal Success
   Synergy: DataCat=0, ResourceCat=0
   Stake Paid: 1000000000000000000 Wei

6. Address: 0xC37D7889ad62de4Fc5798EC7A6fc8d

In [8]:
########################################################################
# 9) Check On-Chain Participant Data
########################################################################
p_count = contract_instance.functions.getParticipantCount().call()
print(f"\nFinal Participant Count on-chain: {p_count}")
print(f"Number of Successful Participants: {len(successful_users)}")
print(f"Number of Failed Attempts: {len(fail_reasons)}\n")

print("On-chain participant data (detailed):\n")
for idx in range(p_count):
    # Retrieve the participant address from the public array
    p_address = contract_instance.functions.participantList(idx).call()
    
    # Fetch their struct from the 'participants' mapping
    part_data = contract_instance.functions.participants(p_address).call()
    # part_data => (bool joined, uint8 dataCat, uint8 resourceCat, uint256 stakePaid, bool isRewarded)

    joined_status = part_data[0]
    data_cat = part_data[1]
    resource_cat = part_data[2]
    stake_paid = part_data[3]
    rewarded_status = part_data[4]

    # Convert numeric categories to more readable labels
    data_cat_label = ["Low", "Medium", "High"][data_cat] if data_cat in [0,1,2] else "InvalidCat"
    resource_cat_label = ["Low", "Medium", "High"][resource_cat] if resource_cat in [0,1,2] else "InvalidCat"

    # Convert stakePaid (Wei) to ETH
    stake_in_eth = w3.from_wei(stake_paid, "ether")

    # Print a summary for this participant
    print(f"Participant #{idx+1} @ {p_address}:")
    print(f"  joined={joined_status}, dataCat={data_cat_label}, resourceCat={resource_cat_label}")
    print(f"  stakePaid={stake_in_eth} ETH, isRewarded={rewarded_status}\n")



Final Participant Count on-chain: 99
Number of Successful Participants: 99
Number of Failed Attempts: 0

On-chain participant data (detailed):

Participant #1 @ 0x85bD17E7a0B9C96dc14470c69d9CeE6617F83aaB:
  joined=True, dataCat=Low, resourceCat=High
  stakePaid=3 ETH, isRewarded=False

Participant #2 @ 0x8E6670a1E36Ba00Bc2b21B6687C54447616A48Ef:
  joined=True, dataCat=Low, resourceCat=Low
  stakePaid=1 ETH, isRewarded=False

Participant #3 @ 0x557598fd2fC58542f34C88746b1A479Ba3d1d753:
  joined=True, dataCat=High, resourceCat=High
  stakePaid=4 ETH, isRewarded=False

Participant #4 @ 0x6653DBeDd9546a8B6c510778caC65b4474A21949:
  joined=True, dataCat=Low, resourceCat=High
  stakePaid=3 ETH, isRewarded=False

Participant #5 @ 0xB3c82BFAb7b450f1B13752071e6AaaB18b84556D:
  joined=True, dataCat=Low, resourceCat=Low
  stakePaid=1 ETH, isRewarded=False

Participant #6 @ 0xC37D7889ad62de4Fc5798EC7A6fc8dfE6764BBDB:
  joined=True, dataCat=High, resourceCat=High
  stakePaid=4 ETH, isRewarded=Fals